In [ ]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

symbol = "EURUSD"
initial_lot_size = 0.01
grid_interval = 20 * mt5.symbol_info(symbol).point
tp_distance = 60 * mt5.symbol_info(symbol).point

def place_initial_orders():
    ask_price = mt5.symbol_info_tick(symbol).ask
    bid_price = mt5.symbol_info_tick(symbol).bid

    buy_stop = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot_size,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": ask_price + grid_interval,
        "tp": ask_price + grid_interval + tp_distance,
        "deviation": 10,
        "magic": 234000,
        "comment": "Initial Buy Stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    sell_stop = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot_size,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": bid_price - grid_interval,
        "tp": bid_price - grid_interval - tp_distance,
        "deviation": 10,
        "magic": 234001,
        "comment": "Initial Sell Stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    mt5.order_send(buy_stop)
    mt5.order_send(sell_stop)

def place_hedge_orders(direction, previous_tp, lot_size):
    if direction == "buy":
        new_sell = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).bid,
            "deviation": 10,
            "magic": 234002,
            "comment": "Hedge Sell",
        }
        new_buy_stop = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": symbol,
            "volume": initial_lot_size,
            "type": mt5.ORDER_TYPE_BUY_STOP,
            "price": previous_tp + grid_interval,
            "deviation": 10,
            "magic": 234003,
            "comment": "New Buy Stop",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        mt5.order_send(new_sell)
        mt5.order_send(new_buy_stop)
    else:
        new_buy = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "deviation": 10,
            "magic": 234004,
            "comment": "Hedge Buy",
        }
        new_sell_stop = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": symbol,
            "volume": initial_lot_size,
            "type": mt5.ORDER_TYPE_SELL_STOP,
            "price": previous_tp - grid_interval,
            "deviation": 10,
            "magic": 234005,
            "comment": "New Sell Stop",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        mt5.order_send(new_buy)
        mt5.order_send(new_sell_stop)

def calculate_breakeven_tp(positions):
    total_volume = sum(pos.volume for pos in positions)
    weighted_sum_price = sum(pos.price_open * pos.volume for pos in positions)
    breakeven_price = weighted_sum_price / total_volume
    return breakeven_price

def manage_trades():
    while True:
        positions = mt5.positions_get(symbol=symbol)
        buy_positions = [pos for pos in positions if pos.type == mt5.ORDER_TYPE_BUY]
        sell_positions = [pos for pos in positions if pos.type == mt5.ORDER_TYPE_SELL]

        if len(buy_positions) == 1 and not sell_positions:
            # Only one buy trade open, check if it hits TP
            buy_position = buy_positions[0]
            if buy_position.tp and mt5.symbol_info_tick(symbol).ask >= buy_position.tp:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": buy_position.volume,
                    "ticket": buy_position.ticket,
                })
                place_initial_orders()
        elif len(sell_positions) == 1 and not buy_positions:
            # Only one sell trade open, check if it hits TP
            sell_position = sell_positions[0]
            if sell_position.tp and mt5.symbol_info_tick(symbol).bid <= sell_position.tp:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": sell_position.volume,
                    "ticket": sell_position.ticket,
                })
                place_initial_orders()
        elif len(buy_positions) > 1:
            # Multiple buy trades open, calculate breakeven TP
            breakeven_tp = calculate_breakeven_tp(buy_positions)
            for pos in buy_positions:
                mt5.order_modify(ticket=pos.ticket, tp=breakeven_tp)
        elif len(sell_positions) > 1:
            # Multiple sell trades open, calculate breakeven TP
            breakeven_tp = calculate_breakeven_tp(sell_positions)
            for pos in sell_positions:
                mt5.order_modify(ticket=pos.ticket, tp=breakeven_tp)
        elif len(buy_positions) == 1 and len(sell_positions) == 1:
            # Both buy and sell trades open, start hedging
            buy_position = buy_positions[0]
            sell_position = sell_positions[0]
            if buy_position.profit >= tp_distance:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": buy_position.volume,
                    "ticket": buy_position.ticket,
                })
                place_hedge_orders("buy", buy_position.tp, sell_position.volume + 0.01)
            elif sell_position.profit >= tp_distance:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": sell_position.volume,
                    "ticket": sell_position.ticket,
                })
                place_hedge_orders("sell", sell_position.tp, buy_position.volume + 0.01)
        time.sleep(60)

# Start the strategy
place_initial_orders()
manage_trades()


In [ ]:

# List of symbols
symbols = ["EURUSD", "GBPUSD", "AUDUSD", "GBPJPY", "USDJPY", "XAUUSD"]

# Set the specific dates and times to place the orders
scheduled_times = [
    datetime(2024, 7, 5, 13, 28, 0),  # First timer
    #datetime(2024, 7, 3, 14, 58, 0),  # Second timer
    #datetime(2024, 7, 3, 18, 58, 0)   # Third timer
]

# Define the lot size and pip parameters
lot = 1.0
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "AUDUSD": 0.0001, "GBPJPY": 0.01, "USDJPY": 0.01, "XAUUSD": 0.1}

In [ ]:
pip install --upgrade numpy

In [2]:
import numpy as np

np.__version__

'2.0.0'

In [1]:
import MetaTrader5 as mt5
import time
from datetime import datetime

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

login = 48438229  # Your account number
password = "Mq3p8!tJ"  # Your account password
server = "HFMarketsGlobal-Demo"  # Your broker's server

# Login to the trading account
if not mt5.login(login, password, server):
    print(f"Failed to connect to account {login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

symbol = "EURUSD"
initial_lot_size = 0.01
grid_interval = 20 * mt5.symbol_info(symbol).point
tp_distance = 60 * mt5.symbol_info(symbol).point

def place_initial_orders():
    ask_price = mt5.symbol_info_tick(symbol).ask
    bid_price = mt5.symbol_info_tick(symbol).bid

    buy_stop = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot_size,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": ask_price + grid_interval,
        "tp": ask_price + grid_interval + tp_distance,
        "deviation": 10,
        "magic": 234000,
        "comment": "Initial Buy Stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    sell_stop = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": initial_lot_size,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": bid_price - grid_interval,
        "tp": bid_price - grid_interval - tp_distance,
        "deviation": 10,
        "magic": 234001,
        "comment": "Initial Sell Stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    result_buy = mt5.order_send(buy_stop)
    if result_buy.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Failed to place buy stop order: {result_buy.comment}")
    
    result_sell = mt5.order_send(sell_stop)
    if result_sell.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Failed to place sell stop order: {result_sell.comment}")

def place_hedge_orders(direction, previous_tp, lot_size):
    if direction == "buy":
        new_sell = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).bid,
            "deviation": 10,
            "magic": 234002,
            "comment": "Hedge Sell",
        }
        new_buy_stop = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": symbol,
            "volume": initial_lot_size,
            "type": mt5.ORDER_TYPE_BUY_STOP,
            "price": previous_tp + grid_interval,
            "deviation": 10,
            "magic": 234003,
            "comment": "New Buy Stop",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result_sell = mt5.order_send(new_sell)
        if result_sell.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Failed to place hedge sell order: {result_sell.comment}")
        
        result_buy_stop = mt5.order_send(new_buy_stop)
        if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Failed to place new buy stop order: {result_buy_stop.comment}")
    else:
        new_buy = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "deviation": 10,
            "magic": 234004,
            "comment": "Hedge Buy",
        }
        new_sell_stop = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": symbol,
            "volume": initial_lot_size,
            "type": mt5.ORDER_TYPE_SELL_STOP,
            "price": previous_tp - grid_interval,
            "deviation": 10,
            "magic": 234005,
            "comment": "New Sell Stop",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result_buy = mt5.order_send(new_buy)
        if result_buy.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Failed to place hedge buy order: {result_buy.comment}")
        
        result_sell_stop = mt5.order_send(new_sell_stop)
        if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Failed to place new sell stop order: {result_sell_stop.comment}")

def calculate_breakeven_tp(positions):
    total_volume = sum(pos.volume for pos in positions)
    weighted_sum_price = sum(pos.price_open * pos.volume for pos in positions)
    breakeven_price = weighted_sum_price / total_volume
    return breakeven_price

def manage_trades():
    while True:
        positions = mt5.positions_get(symbol=symbol)
        buy_positions = [pos for pos in positions if pos.type == mt5.ORDER_TYPE_BUY]
        sell_positions = [pos for pos in positions if pos.type == mt5.ORDER_TYPE_SELL]

        if len(buy_positions) == 1 and not sell_positions:
            # Only one buy trade open, check if it hits TP
            buy_position = buy_positions[0]
            if buy_position.tp and mt5.symbol_info_tick(symbol).ask >= buy_position.tp:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": buy_position.volume,
                    "ticket": buy_position.ticket,
                })
                place_initial_orders()
        elif len(sell_positions) == 1 and not buy_positions:
            # Only one sell trade open, check if it hits TP
            sell_position = sell_positions[0]
            if sell_position.tp and mt5.symbol_info_tick(symbol).bid <= sell_position.tp:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": sell_position.volume,
                    "ticket": sell_position.ticket,
                })
                place_initial_orders()
        elif len(buy_positions) > 1:
            # Multiple buy trades open, calculate breakeven TP
            breakeven_tp = calculate_breakeven_tp(buy_positions)
            for pos in buy_positions:
                mt5.order_modify(ticket=pos.ticket, tp=breakeven_tp)
        elif len(sell_positions) > 1:
            # Multiple sell trades open, calculate breakeven TP
            breakeven_tp = calculate_breakeven_tp(sell_positions)
            for pos in sell_positions:
                mt5.order_modify(ticket=pos.ticket, tp=breakeven_tp)
        elif len(buy_positions) == 1 and len(sell_positions) == 1:
            # Both buy and sell trades open, start hedging
            buy_position = buy_positions[0]
            sell_position = sell_positions[0]
            if buy_position.profit >= tp_distance:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": buy_position.volume,
                    "ticket": buy_position.ticket,
                })
                place_hedge_orders("buy", buy_position.tp, sell_position.volume + 0.01)
            elif sell_position.profit >= tp_distance:
                mt5.order_send({
                    "action": mt5.TRADE_ACTION_CLOSE,
                    "symbol": symbol,
                    "volume": sell_position.volume,
                    "ticket": sell_position.ticket,
                })
                place_hedge_orders("sell", sell_position.tp, buy_position.volume + 0.01)
        time.sleep(60)

# Start the strategy
place_initial_orders()
manage_trades()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\NOC\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\NOC\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\NOC\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\NOC\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\NOC\anaconda3\lib\site-packages\ipyke

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import